In [33]:
import numpy as np
import pandas as pd
import scipy.stats
import statsmodels.stats.multitest as smm

In [15]:
data = pd.read_csv("gene_high_throughput_sequencing.csv")
genes = data.iloc[:, 2:]
print data.shape
data.head()

(72, 15750)


,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [16]:
data["Diagnosis"].value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [27]:
ne = scipy.stats.ttest_ind(genes[data["Diagnosis"] == "normal"], genes[data["Diagnosis"] == "early neoplasia"], axis=0, equal_var=False)
ec = scipy.stats.ttest_ind(genes[data["Diagnosis"] == "early neoplasia"], genes[data["Diagnosis"] == "cancer"], axis=0, equal_var=False)

In [30]:
print "Normal & Early Neoplasia:", sum(ne.pvalue < 0.05)
print "Early Neoplasia & Cancer:", sum(ec.pvalue < 0.05)

Normal & Early Neoplasia: 1575
Early Neoplasia & Cancer: 3490


In [60]:
def fold_change(treatment, control):
    t = np.mean(treatment, axis=0)
    c = np.mean(control, axis=0)
    return [x[0] / x[1] if x[0] > x[1] else -x[1] / x[0] for x in zip(t, c)]

In [61]:
ne_reject, ne_pvalue, _, _ = smm.multipletests(ne.pvalue, alpha=0.025, method='holm')
ec_reject, ec_pvalue, _, _ = smm.multipletests(ec.pvalue, alpha=0.025, method='holm')

In [62]:
nef = sum(np.abs(fold_change(genes[data["Diagnosis"] == "normal"].iloc[: , ne_reject],
                             genes[data["Diagnosis"] == "early neoplasia"].iloc[:, ne_reject])) > 1.5)
ecf = sum(np.abs(fold_change(genes[data["Diagnosis"] == "early neoplasia"].iloc[: , ec_reject],
                             genes[data["Diagnosis"] == "cancer"].iloc[:, ec_reject])) > 1.5)
print "Normal & Early Neoplasia: ", nef
print "Early Neoplasia & Cacner: ", ecf

Normal & Early Neoplasia:  2
Early Neoplasia & Cacner:  77


In [63]:
ne2_reject, ne2_pvalue, _, _ = smm.multipletests(ne.pvalue, alpha=0.025, method='fdr_bh')
ec2_reject, ec2_pvalue, _, _ = smm.multipletests(ec.pvalue, alpha=0.025, method='fdr_bh')

In [64]:
nef2 = sum(np.abs(fold_change(genes[data["Diagnosis"] == "normal"].iloc[: , ne2_reject],
                              genes[data["Diagnosis"] == "early neoplasia"].iloc[:, ne2_reject])) > 1.5)
ecf2 = sum(np.abs(fold_change(genes[data["Diagnosis"] == "early neoplasia"].iloc[: , ec2_reject],
                              genes[data["Diagnosis"] == "cancer"].iloc[:, ec2_reject])) > 1.5)
print "Normal & Early Neoplasia: ", nef2
print "Early Neoplasia & Cacner: ", ecf2

Normal & Early Neoplasia:  4
Early Neoplasia & Cacner:  524
